This example is available as a jupyter notebook [here](https://github.com/SimiPixel/x_xy_v2/blob/main/docs/notebooks/custom_joint_type_dynamics.ipynb).

## Defining a custom Joint Type that supports dynamical simulation

In this notebook we will define a new joint type that is a hinge joint with a random joint axes direction.

It will *also support dynamical simulation*.

In [1]:
import x_xy
from x_xy import maths

import jax
import jax.numpy as jnp

import mediapy as media

We will give this new joint type the identifier `rr` (random revolute). Although it actually already exists in the library, but we can overwrite it.

In [2]:
from x_xy.algorithms.jcalc import _draw_rxyz

# we use such a `params` input to specify the joint-axes, if we later then randomize the attribute of the system object
# we will have the effect of a hinge joint with a randomized joint axes direction
# here we tell the library how it should initialize this `params` PyTree
def _draw_random_joint_axis(key):
    return maths.rotate(jnp.array([1.0, 0, 0]), maths.quat_random(key))

def _rr_init_joint_params(key):
    return dict(joint_axes=_draw_random_joint_axis(key))

# next, we tell the library how it can randomly draw a trajectory for its generalized coordinate; the hinge joint angle
def _rr_transform(q, params):
    # here we use this `params` object
    axis = params["joint_axes"]
    q = jnp.squeeze(q)
    rot = x_xy.maths.quat_rot_axis(axis, q)
    return x_xy.Transform.create(rot=rot)

# this tells the library how to dynamically simulate the type of joint
def _motion_fn(params):
    return x_xy.base.Motion.create(ang=params["joint_axes"])

# now, we can put it all together into a new `x_xy.JointModel`
rr_joint = x_xy.JointModel(_rr_transform, motion=[_motion_fn], rcmg_draw_fn=_draw_rxyz, init_joint_params=_rr_init_joint_params)

# and then we register the joint; Note that `overwrite`=True, because it already exists; that way you can e.g. overwrite the
# default joint types such as the free joint
x_xy.register_new_joint_type("rr", rr_joint, q_width=1, qd_width=1, overwrite=True)

In [3]:
xml_str = """
<x_xy>
    <options gravity="0 0 9.81" dt="0.01"/>
    <worldbody>
        <geom type="xyz" dim="0.1"/>
        <body name="pendulum" joint="rr" pos="0 0 0.5" damping=".01">
            <geom type="xyz" dim="0.1"/>
            <geom type="box" mass="0.5" pos="0.25 0 0" dim="0.5 0.1 0.1"/>
        </body>
    </worldbody>
</x_xy>
"""

# this seed determines (among other things) the randomness of the joint-axes direction
# via the above specified `_rr_init_joint_params`
seed: int = 2
sys = x_xy.load_sys_from_str(xml_str, seed=seed)

In [4]:
state = x_xy.State.create(sys)
xs = []
for t in range(500):
    state = jax.jit(x_xy.step)(sys, state)
    xs.append(state.x)

In [5]:
sys.links.joint_params

{'rr': {'joint_axes': Array([[-0.20673221,  0.39784992,  0.89385533]], dtype=float32)},
 'default': Array([], dtype=float32)}

In [6]:
def show_video(sys, xs: list[x_xy.Transform]):
    if not isinstance(xs, list):
        xs = [xs[t] for t in range(xs.shape())]
    frames = x_xy.render(sys, [xs[i] for i in range(0, len(xs), 4)], camera=None)
    media.show_video(frames, fps=25)

show_video(sys, xs)

Rendering frames..: 100%|██████████| 125/125 [00:00<00:00, 213.26it/s]


This browser does not support the video tag.


the powerful function `x_xy.build_generator` already has the built-in flag `randomize_joint_params` which can be toggled in order to use the user-provided logic `_rr_init_joint_params` for randomizing the joint parameters

In [7]:
xs = x_xy.build_generator(sys, randomize_joint_params=True, _compat=True, aslist=True, eager=True, seed=1, sizes=1)[0][1]

eager data generation: 1it [00:01,  1.26s/it]


In [8]:
show_video(sys, xs)

Rendering frames..: 100%|██████████| 1500/1500 [00:06<00:00, 242.04it/s]


This browser does not support the video tag.


but for `dynamic_simulation` flag to work we additional need to specify the function `x_xy.JointModel.p_control_term`

In [9]:
print(rr_joint.p_control_term)

None


In [10]:
try:
    xs = x_xy.build_generator(sys, randomize_joint_params=True, _compat=True, aslist=True, eager=True, seed=1, sizes=1, dynamic_simulation=True)[0][1]
except NotImplementedError:
    print("NotImplementedError: Please specify `JointModel.p_control_term` for joint type `rr`")

eager data generation: 0it [00:00, ?it/s]

NotImplementedError: Please specify `JointModel.p_control_term` for joint type `rr`
